In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Carregamento e Preparação ---
# IMPORTANTE: Coloque sua planilha 'planilha de testes.csv' na mesma pasta que este script.
file_path = '/kaggle/input/jogos-completo/DB_completo (1).csv'

try:
    df = pd.read_csv(file_path)
    print("--- Análise Inicial do Dataset ---")
    print(f"O dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas.")
    
    # Lidando com valores nulos (se houver)
    df.dropna(subset=['preco_dolar'], inplace=True) # Remove linhas onde o preço é nulo
    for col in ['developers', 'publishers']:
        if col in df.columns:
            df[col] = df[col].fillna('Desconhecido')

    # --- 2. Pré-processamento ---
    print("\n--- Iniciando Pré-processamento ---")
    target = 'preco_dolar'


    # --- NOVO PASSO: Agrupar categorias raras para reduzir a dimensionalidade ---
    print("\n--- Agrupando categorias raras ---")
    
    # Vamos definir um limite. Qualquer publisher com menos de 20 jogos será 'Outro'.
    # Você pode ajustar este número.
    min_count_publisher = 20
    publishers_counts = df['publishers'].value_counts()
    # Pega a lista de publishers que aparecem menos que o limite
    rare_publishers = publishers_counts[publishers_counts < min_count_publisher].index
    # Substitui todos eles por 'Outro'
    df['publishers'] = df['publishers'].replace(rare_publishers, 'Outro')
    
    
    # A mesma lógica para os desenvolvedores
    min_count_developer = 20
    developers_counts = df['developers'].value_counts()
    rare_developers = developers_counts[developers_counts < min_count_developer].index
    df['developers'] = df['developers'].replace(rare_developers, 'Outro')
    
    print("Agrupamento concluído!")
    print("Novos totais de publishers únicos:", df['publishers'].nunique())
    print("Novos totaais de developers únicos:", df['developers'].nunique())
    print("-" * 30)
    
    # Removemos colunas que não ajudam na previsão
    features = df.drop(columns=[target, 'preco_euro', 'gameid', 'title'])

    # Converter variáveis de texto em numéricas para o modelo entender
    X_encoded = pd.get_dummies(features, drop_first=True)
    y = df[target]

    print(f"Dimensões de X após encoding: {X_encoded.shape}")

    # --- 3. Divisão dos Dados em Treino e Teste ---
    # 80% para treinar, 20% para testar
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    print(f"\nDados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

    # --- 4. Treinamento do Modelo Random Forest ---
    print("\n--- Treinando o Modelo Random Forest Regressor ---")
    # n_estimators=100 significa que o modelo é uma "floresta" de 100 "árvores" de decisão.
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    print("Modelo treinado com sucesso!")

    # --- 5. Avaliação do Modelo ---
    print("\n--- Avaliando a Performance do Modelo ---")
    y_pred = rf_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Erro Médio Absoluto (MAE): ${mae:.2f}")
    print(f"R-squared (R²): {r2:.2f}")

    # --- 6. Importância das Variáveis (Feature Importances) ---
    print("\n--- Análise das Variáveis Mais Importantes ---")
    importances = rf_model.feature_importances_
    feature_names = X_encoded.columns
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    
    # Pegando as 15 features mais importantes
    top_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(15)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=top_features, palette='viridis')
    plt.title('Top 15 Variáveis Mais Importantes para Prever o Preço')
    plt.xlabel('Importância Relativa')
    plt.ylabel('Variável')
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print(f"ERRO: O arquivo '{file_path}' não foi encontrado.")
    print("Por favor, certifique-se de que o script Python e o arquivo CSV estão na mesma pasta.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")


ERRO: O arquivo '/kaggle/input/jogos-completo/DB_completo (1).csv' não foi encontrado.
Por favor, certifique-se de que o script Python e o arquivo CSV estão na mesma pasta.
